In [1]:
path_test_folder='project_files/synthetic_data_15_3/test'

In [2]:
from performance_measure import calculate_metrics
from models import lenet,resnet,efficient_net
import config
import torch
from torchvision.transforms import v2
import torch.utils.data as data
from torchvision.datasets import ImageFolder
import os

In [3]:
def get_dataloader(path,data_transform):
    dataset=ImageFolder(path,transform=data_transform)
    dataloader=data.DataLoader(dataset,batch_size=config.batch_size)
    print('Number of points in dataset - ', len(dataset))
    return dataloader

In [4]:
model_dict={'lenet':'lenet','resnet':'resnet','resnet256':'resnet256',#'efficient_net':'enet'
            }

In [9]:
for model in model_dict.keys():
    model_name=model_dict[model]
    print('Model - ',model_name)
    out_dir=os.path.join(f'saved_models',f'{model_name}_saved_model')
    data_transforms=torch.load(os.path.join(out_dir,f'transforms_{model_name}.pt'))
    test_loader=get_dataloader(path_test_folder,data_transforms)
    if(model=='lenet'):
        seq_model = lenet.LeNet(config.num_classes)
    elif(model=='resnet' or model=='resnet256'):
        seq_model = resnet.CustomResNet(config.num_classes).get_model()
    elif(model=='efficient_net'):
        enet_model = efficient_net.CustomEfficientNet(config.num_classes).get_model()
    seq_model.load_state_dict(torch.load(os.path.join(out_dir,f'state_{model_name}.pt'),map_location=torch.device('cpu')))
    #calcualte the metrics 5 times and take the average
    out=calculate_metrics(test_loader,seq_model,5,False)
    #take average of the metrics
    print('*'*50)
    print('Accuracy: {:.2f}%'.format(out['accuracy']),end = ' , ')
    print('Precision: {:.2f}'.format(out['precision']),end = ' , ')
    print('Recall: {:.2f}'.format(out['recall']),end = ' , ')
    print('F1 Score: {:.2f}'.format(out['f1_score']),end = ' , ')
    print('Loss: {:.4f}'.format(out['loss']))
    print('*'*50)
   
    


Model -  lenet
Number of points in dataset -  84
**************************************************
Accuracy: 72.14% , Precision: 0.76 , Recall: 0.71 , F1 Score: 0.73 , Loss: 0.9648
**************************************************
Model -  resnet
Number of points in dataset -  84
**************************************************
Accuracy: 66.90% , Precision: 0.68 , Recall: 0.66 , F1 Score: 0.67 , Loss: 0.9169
**************************************************
Model -  resnet256
Number of points in dataset -  84
**************************************************
Accuracy: 68.57% , Precision: 0.73 , Recall: 0.68 , F1 Score: 0.70 , Loss: 0.7802
**************************************************


In [6]:
#loading the lenet model 
out_dir=os.path.join('saved_models','lenet_saved_model')
data_transforms=torch.load(os.path.join(out_dir,'transforms_lenet.pt'))
test_dataloader=get_dataloader(path_test_folder,data_transforms)
seq_model = lenet.LeNet(config.num_classes)
seq_model.load_state_dict(torch.load(os.path.join(out_dir,'state_lenet.pt'),map_location=torch.device('cpu')))
out_test_lenet=calculate_metrics(test_dataloader,seq_model)

Number of points in dataset -  84
Avergae loss 1.1975 , Accuracy: 71.43% , Macro recall : 0.6965, Macro precision : 0.6857, Macro f1 : 0.6910
